In [8]:
# RunnableLambda는 LangChain 사용자 정의함수를 통합
# RunnableParallel은 여러 체인을 동시에 병렬로 실행하고, 그 결과를 딕셔너리로 통합

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [10]:
#Lambda
from langchain_core.runnables import RunnableLambda
def upper(text):
    return text.upper()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영어로 대답해줘"),
        ("human", "{Q}")
    ]
)

chain1 = prompt | model | StrOutputParser() | RunnableLambda(upper)
chain2 = prompt | model | StrOutputParser() | RunnableLambda(lambda x : x.upper())

output1 = chain1.invoke({ 'Q': "안녕"})
output2 = chain2.invoke({ 'Q': "안녕"})

output1, output2

('HELLO! HOW CAN I ASSIST YOU TODAY?', 'HELLO! HOW CAN I ASSIST YOU TODAY?')

In [11]:
#Parallel
from langchain_core.runnables import RunnableParallel
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요"),
        ("human", "{topic}")
    ]
)
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요"),
        ("human", "{topic}")
    ]
)
optim_chain = optimistic_prompt | model | StrOutputParser()
pessi_chain = pessimistic_prompt | model | StrOutputParser()
parallel_chain = RunnableParallel(
    {
        'optimistic_opinion' : optim_chain,
        'pessimistic_opinion' : pessi_chain
    }
)
output = parallel_chain.invoke({'topic' : "짜장면과 짬뽕"})
import pprint
pprint.pprint(output)

{'optimistic_opinion': '짜장면과 짬뽕은 정말 맛있는 선택이죠! 두 가지 모두 각기 다른 매력을 가지고 있어서, 어떤 '
                       '날에는 짜장면의 달콤한 맛이, 또 어떤 날에는 짬뽕의 매콤한 국물이 그리워질 수 있어요. 친구들과 '
                       '함께 나눠 먹으면 더 맛있고 즐거운 시간이 될 거예요. 그리고 두 가지 모두 다양한 재료가 '
                       '들어가서 영양도 챙길 수 있답니다! 어떤 걸 선택하든 맛있는 한 끼가 될 거예요!',
 'pessimistic_opinion': '짜장면과 짬뽕은 많은 사람들이 좋아하는 음식이지만, 결국에는 그저 일시적인 만족에 불과합니다. '
                        '맛있게 먹고 나면 배가 부르지만, 그 후에는 소화 불량이나 기름진 음식에 대한 후회가 남기 '
                        '마련이죠. 그리고 짜장면이나 짬뽕을 먹는 순간에도, 건강에 좋지 않은 성분들이 몸에 쌓여가고 '
                        '있다는 사실을 잊지 말아야 합니다. 결국, 이런 음식들은 우리의 삶에 작은 즐거움을 줄 수는 '
                        '있지만, 장기적으로는 건강에 해로운 선택일 뿐입니다.'}


In [13]:
from operator import itemgetter
prompt2 = ChatPromptTemplate.from_messages(
    [
        ('system' , '당신은 객관적 AI입니다. 두 가지 의견을 종합해서 사용자에 알려줘'),
        ('human', "낙관적인 d의견 : {optimistic_opinion} \n 비관적인 의견 : {pessimistic_opinion}")
    ]
)
chain3 =(RunnableParallel(
    {
        'optimistic_opinion' : optim_chain,
        'pessimistic_opinion' : pessi_chain
    })
    | prompt2
    | model
    | StrOutputParser()
)
output = chain3.invoke({'topic' : "짜장면과 짬뽕"})

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ('system' , '당신은 객관적 AI입니다. {topic}에 대한 두 가지 의견을 종합하세요'),
        ('human', "낙관적인 d의견 : {optimistic_opinion} \n 비관적인 의견 : {pessimistic_opinion}")
    ]
)

synthesize_chain = (
    {
        'optimistic_opinion' : optim_chain,
        'pessimistic_opinion' : pessi_chain,
        'topic' : itemgetter('topic')
    }
    | synthesize_prompt
    | model
    | StrOutputParser()    
)

output = synthesize_chain.invoke({"topic" : "짜장면과 짬뽕"})
output

'짜장면과 짬뽕에 대한 두 가지 의견을 종합해보면, 이 두 음식은 각각의 매력과 즐거움을 제공하는 동시에, 건강과 관련된 우려도 동반한다는 점이 강조됩니다. \n\n낙관적인 시각에서는 짜장면의 달콤한 맛과 짬뽕의 매콤한 국물이 서로 다른 기분을 충족시켜주며, 친구들과 함께 나누는 즐거움이 큰 장점으로 언급됩니다. 두 가지 음식을 함께 시켜서 맛의 조화를 즐기는 것도 긍정적인 경험으로 여겨집니다.\n\n반면, 비관적인 시각에서는 이러한 음식들이 일시적인 만족을 줄 뿐, 소화 불량이나 체중 증가와 같은 부작용을 초래할 수 있으며, 건강에 미치는 부정적인 영향이 우려된다고 지적합니다. \n\n결론적으로, 짜장면과 짬뽕은 순간의 즐거움을 제공하지만, 그에 따른 건강상의 고려도 필요하다는 점에서 균형 잡힌 접근이 중요하다고 할 수 있습니다.'